# Proyecto Integrador: Sistema de análisis de ventas

Este notebook muestra cómo aplicar patrones de diseño en Python para interactuar con una base de datos MySQL, realizar consultas avanzadas y manipular los resultados de manera eficiente.

**Requisitos previos:**
- Tener configurado el archivo `.env` con los datos de conexión.
- Haber ejecutado los scripts de creación de tablas y carga de datos, especificamente 'load_data.sql' dentro de la carpeta 'sql'.
- Instalar las dependencias indicadas en el README.

---

# PI: Parte 2

Este notebook muestra cómo utilizar patrones de diseño en Python para interactuar con una base de datos MySQL, realizar consultas avanzadas y manipular los resultados de manera eficiente.

**Contenidos:**
- Conexión a la base de datos (Singleton)
- Ejecución de queries y conversión a objetos
- Uso de Builder Pattern para consultas dinámicas

In [1]:
# Configuración inicial y carga de módulos

import sys
from pathlib import Path
import os

# Agregar src al path para importar como paquete
sys.path.append(str(Path().resolve()))

from src.patterns.singleton.db_connector import MySQLConnector
from src.patterns.builder_pattern.sales_query_builder import SalesQueryBuilder
from src.patterns.factories.sale_factory import Salefactory

In [9]:
# Conexión a la base de datos usando Singleton

db = MySQLConnector()
db.conectar()

Usando instancia existente.
Conexion exitosa a la base de datos 'sales_accenture'


In [3]:
# Ejecutar una consulta simple y mostrar los primeros resultados

query = "SELECT * FROM sales LIMIT 5;"
df = db.query_df(query)
display(df)  # Usar display para mejor visualización en Jupyter

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,14,7,33133,406,9,0.0,63.0,1 days 15:52:00,VOP9A7Y4C5XSM2LLT0UJ\r
1,170,10,19039,377,5,0.0,50.0,1 days 09:40:00,VIIKLFPVD2D5U5E3NKKA\r
2,416,16,71212,351,19,0.0,304.0,1 days 16:57:00,QFA3F3LANPC6UVQBPYL2\r
3,486,23,93253,417,24,0.0,552.0,2 days 11:29:00,6H8SDUI9MCA1IXZ4UCUE\r
4,659,13,65151,147,17,0.2,221.0,1 days 03:58:01,1O062RUFV0LVAQ94G38J\r


In [4]:
# Verificar el tipo de objeto retornado

type(df)

pandas.core.frame.DataFrame

In [5]:
# Instanciar objetos Sale usando el Factory Method

sales_objs = [Salefactory.from_series(row) for _, row in df.iterrows()]
sales_objs[0].__dict__

{'sales_id': 14,
 'sales_person_id': 7,
 'customer_id': 33133,
 'product_id': (406,),
 'quantity': (9,),
 'discount': 0.0,
 'total_price': 63.0,
 'sales_date': Timedelta('1 days 15:52:00'),
 'transaction_number': 'VOP9A7Y4C5XSM2LLT0UJ\r'}

In [6]:
# Usar el Builder Pattern para crear una consulta dinámica
# Ejemplo: Filtrar ventas de un cliente específico con cantidad mínima

builder = (
    SalesQueryBuilder().with_customer(82723).with_min_quantity(10)
)

query_filtrada = builder.build()
print("Consulta generada:", query_filtrada)

df_filtrada = db.query_df(query_filtrada)
display(df_filtrada)

Consulta generada: SELECT * FROM sales WHERE CustomerID = 82723 AND Quantity >= 10


,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,2982027,18,82723,440,21,0.0,378.0,0 days,D5GTTK1ANDEEGCD7XDF2\r


In [7]:
# Convertir resultados filtrados a objetos Sale

ventas_filtradas = [Salefactory.from_series(row) for _, row in df_filtrada.iterrows()]

for venta in ventas_filtradas[:3]:
    print(f"Venta ID: {venta.sales_id}, Monto: {venta.total_price}, ¿Alto Valor?: {venta.is_high_value()}")

Venta ID: 2982027, Monto: 378.0, ¿Alto Valor?: False


In [8]:
# Cerrar la conexión cuando no se necesite más
db.cerrar()

Conexion cerrada.


# PI: Parte 3

## Consultas avanzadas: CTE y funciones de ventana

En esta sección se utilizan CTEs (Common Table Expressions) y funciones de ventana para realizar análisis avanzados sobre los datos de ventas.

**Contenidos:**
- Consultas avanzadas: CTE y funciones de ventana
- Creación y uso de vistas, funciones, procedimientos y triggers
- Optimización de consultas con índices

In [12]:
# Acumulado de ventas por categoría usando Window Function

query_window = SalesQueryBuilder().with_top_acumulado_por_categoria().build()
df_windows = db.query_df(query_window)
print("Acumulado de ventas por categoria:")
print(df_windows.head())

Acumulado de ventas por categoria:
  CategoryName  total_sale  categoria_rank
0  Confections    998526.0               1
1         Meat    862283.0               2
2      Poultry    815645.0               3
3      Cereals    805381.0               4
4      Produce    736564.0               5


In [13]:
# Acumulado de ventas por producto usando Window Function

query_window = SalesQueryBuilder().with_top_acumulado_por_producto().build()
df_windows = db.query_df(query_window)
print("Acumulado de ventas por producto:")
print(df_windows.head())

Acumulado de ventas por producto:
                       ProductName  total_sale  product_rank
0        Coconut - Shredded; Sweet     23871.0             1
1    Soup - Campbells; Beef Barley     23726.0             2
2         Garlic - Primerba; Paste     23566.0             3
3  Bread - Italian Roll With Herbs     23074.0             4
4                     Brandy - Bar     22977.0             5


In [14]:
# Usar CTE para filtrar clientes con ventas totales mayores a 1000

query_cte = SalesQueryBuilder().with_ventas_cte(min_total=1000).build()
df_cte = db.query_df(query_cte)
print("Clientes con ventas totales mayores a 1000:")
print(df_cte)


Clientes con ventas totales mayores a 1000:
     CustomerID   total
0         79900  1407.0
1         67814  1080.0
2         97386  1325.0
3         80634  1176.0
4         97638  1275.0
..          ...     ...
147       95965  1100.0
148       95617  1000.0
149       81586  1050.0
150       96141  1000.0
151       88360  1035.0

[152 rows x 2 columns]


In [15]:
# Usar Window Function para acumulado de ventas por cliente

query_window = SalesQueryBuilder().with_acumulado_por_cliente().build()
df_windows = db.query_df(query_window)
print("Acumulado de ventas por cliente:")
print(df_windows.head())

Acumulado de ventas por cliente:
   CustomerID       SalesDate  TotalPrice  acumulado_cliente
0           1 0 days 23:37:01         2.0                2.0
1           1 2 days 07:10:00        19.0               21.0
2           4 1 days 04:56:00        20.0               20.0
3           8 0 days 11:43:00        20.0               20.0
4          10 0 days 15:16:00        14.0               14.0


# Creación y uso de vistas, funciones y procedimientos

En esta sección se ejecutan scripts SQL para crear vistas, funciones, procedimientos y triggers en la base de datos. Esto permite extender la funcionalidad y optimizar consultas.

In [16]:
# Crear vista de ventas por cliente
# Si ocurre un error de SQL, se mostrará la sentencia problemática

ruta = os.path.join('sql', 'view_sales_customer.sql')
try:
    db.ejecutar_script_sql(ruta)
except Exception as e:
    print("Error al ejecutar el script:", e)

Script 'sql\view_sales_customer.sql' ejecutado correctamente.


In [17]:
# Crear vista de ventas por empleado y mes

ruta2 = os.path.join('sql', 'view_sales_by_employee_month.sql')
db.ejecutar_script_sql(ruta2)

Script 'sql\view_sales_by_employee_month.sql' ejecutado correctamente.


In [18]:
# Consultar la vista creada

query = "SELECT * FROM vista_ventas_clientes LIMIT 5;"
df = db.query_df(query)
print(df)

   SalesID  TotalPrice       SalesDate FirstName LastName
0       14        63.0 1 days 15:52:00   Lakesha    Young
1      170        50.0 1 days 09:40:00      Josh   Horton
2      416       304.0 1 days 16:57:00      Ivan   Campos
3      486       552.0 2 days 11:29:00     Grant      Day
4      659       221.0 1 days 03:58:01     Devin  Herring


In [19]:
# Crear función para calcular edad

ruta3 = os.path.join('sql', 'function_calculate_age.sql')
db.ejecutar_script_sql(ruta3)

Script 'sql\function_calculate_age.sql' ejecutado correctamente.


In [ ]:
# Crear procedimiento almacenado para empleados por rango de edad

ruta4 = os.path.join('sql', 'procedure_get_employees_by_age_range.sql')
db.ejecutar_script_sql(ruta4)

In [21]:
# Crear indices 

ruta5 = os.path.join('sql', 'index.sql')
db.ejecutar_script_sql(ruta5)

Script 'sql\index.sql' ejecutado correctamente.


In [ ]:
# Ejemplo de scripts SQL para crear objetos en la base de datos

sql_objects = """

-- 1. Vista: Resumen de ventas por empleado y mes
CREATE OR REPLACE VIEW sales_by_employee_month AS
SELECT
    e.EmployeeID,
    CONCAT(e.FirstName, ' ', e.LastName) as EmployeeName,
    YEAR(s.SalesDate) as Year,
    MONTH(s.SalesDate) as Month,
    COUNT(*) as TotalTransactions,
    SUM(s.Quantity) as TotalQuantity,
    SUM(s.TotalPrice) as TotalSales
FROM sales s
JOIN employees e ON s.SalesPersonID = e.EmployeeID
GROUP BY e.EmployeeID, EmployeeName, YEAR(s.SalesDate), MONTH(s.SalesDate)
ORDER BY Year, Month, TotalSales DESC;

-- 2. Funcion: calcular edad a partir de fecha de nacimiento
DELIMITER // 
CREATE FUNCTION calculate_age(birth_date DATETIME)
RETURNS INT
DETERMINISTIC
BEGIN
    RETURN TIMESTAMPDIFF(YEAR, birth_date, CURDATE());
END //
DELIMITER ;

-- 3. Procedimiento almacenado: obtener empleados por rango de edad
DELIMITER //
CREATE PROCEDURE get_employees_by_age_range(IN min_age INT, IN max_age INT)
BEGIN
    SELECT
        EmployeeID,
        CONCAT(FirstName, ' ', LastName) as EmployeeName,
        calculate_age(BirthDate) as Age,
        Gender,
        HireDate
    FROM employees
    WHERE calculate_age(BirthDate) BETWEEN min_age AND max_age
    ORDER BY Age;
END //
DELIMITER ;

-- 4. Trigger: Registrar fecha de modificacion de productos
DELIMITER //
CREATE TRIGGER before_product_update
BEFORE UPDATE ON products
FOR EACH ROW
BEGIN
    SET NEW.ModifyDate = CURRENT_TIME();
END // 
DELIMITER ;

-- 5. Indice compuesto para mejorar consultas de ventas
CREATE INDEX idx_sales_date_product ON sales(SalesDate, ProductID);
CREATE INDEX idx_sales_customer_date ON sales(CustomerID, SalesDate);

"""

# Extra:

In [ ]:
# Comparación de tiempos de consulta antes y después de crear índices

import time

print("Ejecutando consulta sin optimizar ...")
start = time.time()
db.query_df(query_slow)
slow_time = time.time() - start
print(f"Tiempo sin indice: {slow_time:.4f} segundos")

#CREATE INDEX idx_sales_date_product ON sales(SalesDate, ProductID);
#CREATE INDEX idx_sales_customer_date ON sales(CustomerID, SalesDate);

print("Ejecutando consulta optimizada...")
start = time.time()
db.query_df(query_fast)
fast_time = time.time() - start
print(f"Tiempo con indice: {fast_time:.4f} segundos")
print(f"Mejora: {((slow_time - fast_time) / slow_time * 100):.2f}% más rápido")